## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,2022-05-17 13:05:06,-46.1927,168.8643,45.48,92,94,3.15,overcast clouds
1,1,Bredasdorp,ZA,2022-05-17 13:05:07,-34.5322,20.0403,80.87,24,21,6.93,few clouds
2,2,Roma,US,2022-05-17 12:54:03,43.2128,-75.4557,52.95,94,98,4.00,overcast clouds
3,3,Karratha,AU,2022-05-17 13:05:07,-20.7377,116.8463,72.59,89,99,5.73,light rain
4,4,Carnarvon,AU,2022-05-17 13:05:08,-24.8667,113.6333,72.30,61,0,8.46,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Bredasdorp,ZA,2022-05-17 13:05:07,-34.5322,20.0403,80.87,24,21,6.93,few clouds
12,12,Queimadas,BR,2022-05-17 13:05:12,-10.9783,-39.6264,84.11,56,93,3.89,overcast clouds
14,14,San Policarpo,PH,2022-05-17 13:05:12,12.1791,125.5072,81.73,82,96,10.49,overcast clouds
15,15,Butaritari,KI,2022-05-17 13:05:13,3.0707,172.7902,81.64,76,20,16.40,few clouds
20,20,Rikitea,PF,2022-05-17 13:05:14,-23.1203,-134.9692,76.37,79,15,21.65,few clouds
22,22,Victoria,HK,2022-05-17 13:04:26,22.2855,114.1577,75.18,75,74,13.00,broken clouds
25,25,Manicaragua,CU,2022-05-17 13:05:16,22.1494,-79.9736,79.25,77,74,2.15,broken clouds
27,27,Katakwi,UG,2022-05-17 13:05:16,1.8911,33.9661,80.96,50,69,6.44,broken clouds
29,29,Hithadhoo,MV,2022-05-17 13:05:17,-0.6000,73.0833,84.16,76,56,18.54,broken clouds
33,33,Atuona,PF,2022-05-17 13:05:18,-9.8000,-139.0333,78.87,66,93,17.65,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                235
City                   235
Country                233
Date                   235
Lat                    235
Lng                    235
Max Temp               235
Humidity               235
Cloudiness             235
Wind Speed             235
Current Description    235
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bredasdorp,ZA,80.87,few clouds,-34.5322,20.0403,
12,Queimadas,BR,84.11,overcast clouds,-10.9783,-39.6264,
14,San Policarpo,PH,81.73,overcast clouds,12.1791,125.5072,
15,Butaritari,KI,81.64,few clouds,3.0707,172.7902,
20,Rikitea,PF,76.37,few clouds,-23.1203,-134.9692,
22,Victoria,HK,75.18,broken clouds,22.2855,114.1577,
25,Manicaragua,CU,79.25,broken clouds,22.1494,-79.9736,
27,Katakwi,UG,80.96,broken clouds,1.8911,33.9661,
29,Hithadhoo,MV,84.16,broken clouds,-0.6000,73.0833,
33,Atuona,PF,78.87,overcast clouds,-9.8000,-139.0333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bredasdorp,ZA,80.87,few clouds,-34.5322,20.0403,Bredasdorp Country Manor
12,Queimadas,BR,84.11,overcast clouds,-10.9783,-39.6264,Hotel Mandala
14,San Policarpo,PH,81.73,overcast clouds,12.1791,125.5072,Agapita M. Nicart
15,Butaritari,KI,81.64,few clouds,3.0707,172.7902,Isles Sunset Lodge
20,Rikitea,PF,76.37,few clouds,-23.1203,-134.9692,People ThankYou


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))